In [2]:
import cvxpy as cp
import pandas as pd
import numpy as np
from vizlib import *

In [125]:
# Import data
avail = pd.read_csv('availability.csv')
staff_data = pd.read_csv('staff.csv')
names = avail.T.index
staff = staff_data[staff_data.person == names]

# Sets
num_weeks, num_people = avail.shape
people = range(0,num_people)
weeks = range(0,num_weeks)
is_resident = staff.resident
res = is_resident[is_resident == 1].index
path = is_resident[is_resident == 0].index

# Paramaters
n_target = (staff.n_target * num_weeks).to_numpy()
b_target = (staff.b_target * num_weeks).to_numpy()

n_rp_freq = n_target[res]/len(path)
n_rp_target = np.tile(n_rp_freq.reshape((-1,1)), len(path))

b_rp_freq = b_target[res]/len(path)
b_rp_target = np.tile(b_rp_freq.reshape((-1,1)), len(path))

# Decision variables
n = cp.Variable(avail.shape, boolean=True, name='necropsies')
b = cp.Variable(avail.shape, boolean=True, name='biopsies')

# Definition variables
c = n + b
constraints = []

# Constraint: Each duty requires one pathologist
c_one_path_n = [cp.sum(n[w,path]) == 1 for w in weeks]
c_one_path_b = [cp.sum(b[w,path]) == 1 for w in weeks]
constraints += c_one_path_n + c_one_path_b

# Constraint: Each duty can have a max of one resident
c_one_res_n = [cp.sum(n[w,res]) <= 1 for w in weeks]
c_one_res_b = [cp.sum(b[w,res]) <= 1 for w in weeks]
constraints += c_one_res_n + c_one_res_b

# Constraint: Cannot work both duties or during time off
c_work_avail = [c[w,p] <= avail.iloc[w,p] for w in weeks for p in people]
constraints += c_work_avail

# Cost: Two consecutive duties excluding NB
nn_first = cp.Variable(avail.shape, boolean=True)
nn_second = cp.Variable(avail.shape, boolean=True)
bb_first = cp.Variable(avail.shape, boolean=True)
bb_second = cp.Variable(avail.shape, boolean=True)
bn_first = cp.Variable(avail.shape, boolean=True)
bn_second = cp.Variable(avail.shape, boolean=True)

c_nn_doubles = [nn_first[w] + nn_second[w] == cp.sum(n[w:w+2], axis=0) for w in weeks]
c_bb_doubles = [bb_first[w] + bb_second[w] == cp.sum(b[w:w+2], axis=0) for w in weeks]
c_bn_doubles = [bn_first[w] + bn_second[w] == b[w] + n[w+1] for w in weeks[:-1]]
constraints += c_nn_doubles + c_bb_doubles + c_bn_doubles

# Cost: Three consecutive duties
triple_one_two = cp.Variable(avail.shape, integer=True)
triple_third = cp.Variable(avail.shape, boolean=True)
c_triples = [triple_one_two[w] + triple_third[w] == cp.sum(c[w:w+3], axis=0) for w in weeks]
constraints += c_triples + [triple_one_two <= 2]

# Cost: Absolute difference between target Bx and scheduled Bx
b_diff = cp.Variable(num_people)
b_below = cp.Variable(num_people, nonneg=True)
b_above = cp.Variable(num_people, nonneg=True)

c_b_diff = [b_diff == cp.sum(b, axis=0) - b_target]
c_b_below = [b_below >= cp.neg(b_diff)]
c_b_above = [b_above >= b_diff]
constraints += c_b_diff + c_b_below + c_b_above

# Cost: Absolute difference between target Nx and scheduled Nx
n_diff = cp.Variable(num_people)
n_below = cp.Variable(num_people, nonneg=True)
n_above = cp.Variable(num_people, nonneg=True)

c_n_diff = [n_diff == cp.sum(n, axis=0) - n_target]
c_n_below = [n_below >= cp.neg(n_diff)]
c_n_above = [n_above >= n_diff]
constraints += c_n_diff + c_n_below + c_n_above

# Cost: Shortage of target Nx R/P pairs
n_pairs = [cp.Variable((len(res), len(path)), boolean=True) for _ in weeks]
n_pairs_below = cp.Variable((len(res), len(path)), nonneg=True)

c1_n_pairs = [n_pairs[w][ir,ip] <= n[w,p] for ip,p in enumerate(path) for ir,_ in enumerate(res) for w in weeks]
c2_n_pairs =[n_pairs[w][ir,ip] <= n[w,r] for ip,_ in enumerate(path) for ir,r in enumerate(res) for w in weeks]
c3_n_pairs = [n_pairs_below >= n_rp_target - cp.sum(n_pairs, axis=2)]
constraints += c1_n_pairs + c2_n_pairs + c3_n_pairs

# Cost: Shortage of target Bx R/P pairs
b_pairs = [cp.Variable((len(res), len(path)), boolean=True) for _ in weeks]
b_pairs_below = cp.Variable((len(res), len(path)), nonneg=True)

c1_b_pairs = [b_pairs[w][ir,ip] <= b[w,p] for ip,p in enumerate(path) for ir,_ in enumerate(res) for w in weeks]
c2_b_pairs =[b_pairs[w][ir,ip] <= b[w,r] for ip,_ in enumerate(path) for ir,r in enumerate(res) for w in weeks]
c3_b_pairs = [b_pairs_below >= b_rp_target - cp.sum(b_pairs, axis=2)]
constraints += c1_b_pairs + c2_b_pairs + c3_b_pairs

# Define costs
double_cost = 3*cp.sum(nn_second) + 2*cp.sum(bn_second) + cp.sum(bb_second)
triple_cost = cp.sum(triple_third)
duty_cost = cp.sum(b_below + b_above + n_below + n_above)
pair_cost = cp.sum(n_pairs_below + b_pairs_below)

# Solve
obj = cp.Minimize(double_cost + triple_cost + duty_cost + pair_cost)
prob = cp.Problem(obj, constraints)
prob.solve(solver='GUROBI')
viz_schedule(avail, b.value, n.value)

,NM,MP,SA,LKC,DG,MC,AK,CC,KB
0,,B,N,-,,,-,B,N
1,B,-,-,-,N,B,N,,-
2,,N,-,-,B,,B,N,-
3,N,-,-,-,B,,,B,N
4,B,N,-,-,,N,B,-,
5,,B,,-,N,,-,-,N
6,B,-,N,-,-,N,,,B
7,N,-,B,-,-,B,N,,-
8,,N,-,B,-,,B,,-
9,N,B,-,-,,,-,N,B
